In [ ]:
from __future__ import annotations
import numpy as np
import asyncio
import aiohttp
import requests
import wave
import json
import os
import librosa
from IPython.display import Audio

In [ ]:
HOST = ''
TOKEN = ''
HEADER = {
    'procon-token': TOKEN
}

In [ ]:
async def get_match():
    # problems: 試合中の問題数。整数
    # bonus_factor: 使用した分割数に対するボーナス係数の配列。n番目がn個利用場合のボーナス係数。
    # penalty: 変更札に適用される係数。整数

    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(HOST + '/match', headers=HEADER, ssl=True) as res:
                if res.status == requests.codes.ok:
                    data = await res.json()
                    success = True
                else:
                    data = await res.text()
                    success = False
                return (success, data)
        except Exception as err:
            print(err)

In [ ]:
async def get_problem():
    # id: 問題ID
    # chunks: 分割数。整数
    # starts_at: 開始時間のunixtime
    # time_limit: 制限時間。単位は秒。整数
    # data: 重ね合わせ数。整数
    
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(HOST + '/problem', headers=HEADER, ssl=True) as res:
                if res.status == requests.codes.ok:
                    data = await res.json()
                    success = True
                else:
                    data = await res.text()
                    success = False
                return (success, data)
        except Exception as err:
            print(err)

In [ ]:
async def get_chunk(n_chunk):
    # chunks: 各分割データのファイル名の配列

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(HOST + '/problem/chunks?n=' + str(n_chunk), headers=HEADER, ssl=True) as res:
                if res.status == requests.codes.ok:
                    data = await res.json()
                    success = True
                else:
                    data = await res.text()
                    success = False
                return (success, data)
        except Exception as err:
            print(err)

In [ ]:
async def get_file(file_name, save_path):
    
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(HOST + '/problem/chunks/' + file_name, headers=HEADER, ssl=True) as res:
                if res.status == requests.codes.ok:
                    data = await res.read()
                    success = True
                    with wave.open(save_path + file_name, mode='wb') as f:
                        f.setnchannels(1)
                        f.setsampwidth(2)
                        f.setframerate(48000)
                        f.writeframes(bytes(data))
                else:
                    data = await res.text()
                    success = False
                return (success, data)
        except Exception as err:
            print(err)

In [ ]:
async def answer(problem_id, answers):
    # problem_id: 問題ID
    # answers: 解凍された絵札のIDの配列
    # accepted_at: 回答を受信した日時のunix time
    body = json.dumps({'problem_id': problem_id, 'answers': answers})

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post(HOST + '/problem', data=body, headers=HEADER, ssl=True) as res:
                if res.status == requests.codes.ok:
                    data = await res.json()
                    success = True
                else:
                    data = await res.text()
                    success = False
                return (success, data)
        except Exception as err:
            print(err)

In [ ]:
problem = await get_problem()
print(problem)

In [ ]:
print(problem[1]['id'])

In [ ]:
chunks = await get_chunk(1)
print(chunks)
file_name1 = chunks[1]["chunks"][0]
file_name2 = problem[1]['id'] + ".wav"
print(file_name1)
print(file_name2)

In [ ]:
file_dir = "../problems/"
sound = await get_file(file_name1, file_dir)
os.rename(file_dir + file_name1, file_dir + file_name2)
print(sound[0])

In [ ]:
sound_array = np.frombuffer(sound[1], dtype=np.int16)
Audio(file_dir + file_name2, rate=sound_array[1])
# print(sound_array[1])

In [ ]:
# {
#     'get_match': (get_match),
#     'get_problem': (get_problem),
#     'get_chunk': (get_chunk),
#     'get_file': (get_file),
#     'submit answer': (answer),
#     'predict': ["01", "02", "03"]
# }